# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Bidirectional Brute Force with Wildcards</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Code modified from: https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import os
import time


import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
import random
from sympy.combinatorics import Permutation
from typing import Dict, List
import zipfile
import numpy as np
import sqlite3

In [2]:
database_file = '../solutions.db'
solution_method = 'bidirectional brute force w/ wildcards'
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

select_query = "SELECT id FROM solutions WHERE solution_method <> ? ORDER BY count"
    
# Execute the query
cursor.execute(select_query, (solution_method,))
unsolved_ids = cursor.fetchall()


# Commit the changes and close the connection
conn.commit()
conn.close()



RUN_TIME = 1 #60 * 60 * 24 * 7
START_TIME = time.time()
TIMEOUT = RUN_TIME

INCLUDES = [id[0] for id in unsolved_ids]
INCLUDES = [206]


# INCLUDES = list(range(333, 338))

EXCLUDES = []
MIN_SIZE = None
MAX_SIZE = None

DEBUG = False


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [3]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [4]:


# puzzle = puzzles.loc[id]
# sol = brute_force.solve(puzzle, puzzle_info, ';'.join(state))
# sol

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [5]:
import importlib
import brute_force
importlib.reload(brute_force);

moves = None
source_paths = None
dest_paths = None

database_file = '../solutions.db'
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

grows = []
all_ids = submission['id'].unique()
for pid in INCLUDES:
        
    df = puzzles.loc[pid]
    initial_state = df['initial_state']
    solution_state = df['solution_state']
    size = len(solution_state.split(';'))
    
    puzzle_type = df['puzzle_type']
    num_wildcards = df['num_wildcards']
    allowed_moves = puzzle_info.loc[puzzle_type, 'allowed_moves']
    solution = brute_force.solve(
        num_wildcards,
        allowed_moves,
        ';'.join(initial_state),
        ';'.join(solution_state))


    if solution is None:
        print('=> [' + str(pid) + '] Failed!')        
    else:
        print('=> [' + str(pid) + '] Success!')        
        
        grows.append({'id': pid, 'moves': solution})
        
        
        game_id = int(pid)
        moves = solution
        move_count = len(moves.split('.'))
        
        select_query = "SELECT count FROM solutions WHERE id = ?"
            
        # Execute the query
        cursor.execute(select_query, (game_id,))
        best_move_count = cursor.fetchone()
        
        if best_move_count[0] > move_count:
            print(f'Improvement to {game_id}')
            # Insert the moves into the database
            insert_query = "INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)"
            cursor.execute(insert_query, (game_id, moves, move_count, solution_method))
            conn.commit()
        

# Commit the changes and close the connection
conn.commit()
conn.close()

1
2
3
4
5
6
7
8
9
10
11


KeyboardInterrupt: 

In [6]:
# counter clockwise
corners_2x2x2 = [
    (0, 13, 16),
    (1, 9, 12),
    (2, 17, 4),
    (3, 5, 8),
    (20, 6, 19),
    (21, 10, 7),
    (22, 18, 15),
    (23, 14, 11),
]

In [79]:
initial_state = df['initial_state']
s = df['initial_state'].split(';')
staging_moves = ("-f2.-f3.-f2.-f3.-d2.-d3.-d2.-d3.-f3.-f3.d0.d1.r0.-f2.-f3.-f2.-f3.-d0.-f0.-f1.-d3.r2.r3.-r0.d0.d1.d0"
                 ".d1.r0.r1.r0.r1.-d3.r3.d0.d1.d0.d1.d2.d3.f0.f1.f0.f1.-f2.-f3.-f2.-f3.d0.d1.r0.r1.r0.r1.-d3.d0.d1.d0"
                 ".d1.f0.d0.f0.f1.f0.f1.r0.r1.r0.r1.f3.-r3.-r3.d0.-f3.-f3.-d0.-r3.-r3.r0.r1.r0.r1.d3.f0.f1.f0.f1"
                 ".-d3.f3.f3.r3"
                 ".f0.f0.r0"
                 ".-r3"
                 ".d1.d1.r0.r0.f1.f1.r3.d2.d2.r3.d2.d2.r0.r0.f2.f2"
                 # ".r3.r3"
                 ".-r3"
                 ".d1.d1.r0.r0.f1.f1.r3.d2.d2.r3.d2.d2.r0.r0.f2.f2"
                 ".r3.r3"
                 # # ".-d0"
                 # ".f1.f1.d3.d3.r2.r2.-d0.f2.f2.-d0.f2.f2.d3.d3.r2.r2"
                 # # ".-d0.-d0"
                 # ".-f0.-f0"
                 ".d1.d1.f0.f0.r2.r2.f3.d2.d2.f3.d2.d2.f3.f3.r2.r2"
                 ".-f0"
                 ".d1.d1.f0.f0.r2.r2.f3.d2.d2.f3.d2.d2.f3.f3.r2.r2"
                 ".-f0"
                 ".-d0"
                 # ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r1.r1" # switch faces
                 # ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r1.r1" # switch faces
                 ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r2.r2" # no switch faces
                 ".-d3.d0"
                 ".f1.f1.d0.d0.r1.r1.d3.f2.f2.d3.f2.f2.d0.d0.r2.r2" # no switch faces
                 ".d0.d0"
                 )
moves = literal_eval(puzzle_info.loc[puzzle_type, 'allowed_moves'])
moves = brute_force.init_reverse_moves(moves)
for idx, move in enumerate(staging_moves.split('.')):
    s = list(np.asarray(s)[np.array(moves[move])])


ss = solution_state.split(';')

# find all corners
for c in corners_2x2x2:
    print([int(ss[f][1:]) for f in c])
        

[0, 13, 16]
[1, 9, 12]
[2, 17, 4]
[3, 5, 8]
[20, 6, 19]
[21, 10, 7]
[22, 18, 15]
[23, 14, 11]


In [80]:
dim = 4
dim_2 = dim * dim
ins = s #initial_state.split(';')
ss = solution_state.split(';')
ins_2x2x2 = []
ss_2x2x2 = []
map = {}
inverse_map = {}
count = 0

for f in range(6):
    for i in range(dim - 2):
        for j in range(dim - 2):
            init_value = ins[f * dim_2 + (i + 1) * dim + j + 1]
            value = ss[f * dim_2 + (i + 1) * dim + j + 1]
            map[value] = f'N{count}'
            inverse_map[count] = value
            
            ins_2x2x2.append(init_value)
            ss_2x2x2.append(value)
            
            # print(f'N{count}')
            count += 1

';'.join(ss_2x2x2)
';'.join(ins_2x2x2)

'N5;N6;N9;N10;N21;N22;N25;N26;N37;N38;N41;N42;N53;N54;N57;N58;N69;N70;N73;N74;N85;N86;N89;N90'

In [81]:
# # verify corners
# # c = corners_2x2x2[5]
# # print(inverse_map[c[0]], inverse_map[c[1]], inverse_map[c[2]])
# for c in corners_2x2x2:
#     # for idx in c:
#     #     print(f'N{idx}')
#     #     print(inverse_map[idx])
#        
#     # print(c[0], c[1], c[2]) 
#     print(inverse_map[c[0]], inverse_map[c[1]], inverse_map[c[2]])
#     print(ins_2x2x2[c[0]], ins_2x2x2[c[1]], ins_2x2x2[c[2]])
#     print('---')
    
print(ins_2x2x2[8:12])
print(ins_2x2x2[16:20])
ins_2x2x2[:4]
ins_2x2x2[20:]
print(ins_2x2x2[4:8])
print(ins_2x2x2[12:16])

['N37', 'N38', 'N41', 'N42']
['N69', 'N70', 'N73', 'N74']
['N21', 'N22', 'N25', 'N26']
['N53', 'N54', 'N57', 'N58']


In [82]:
print(ins_2x2x2[:4])
print(ins_2x2x2[20:])

['N5', 'N6', 'N9', 'N10']
['N85', 'N86', 'N89', 'N90']
